In [2]:
import nbformat
import os
import openai
from dotenv import load_dotenv
import time
import re
import logging
from datetime import datetime

# Load Environment Variables
load_dotenv(".env")
openai.api_key = os.environ["OPENAI_API_KEY"]



#setting up logging
if not logging.getLogger().hasHandlers():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize the OpenAI client
client = openai.OpenAI()
model = 'gpt-4o-mini'


**create an assistant** 

Only needed once


In [3]:
# Check if assistant_id and thread_id exist
if os.path.exists('assistant_thread_ids.txt'):
    # Read assistant_id and thread_id from the file
    with open('assistant_thread_ids.txt', 'r') as f:
        lines = f.readlines()
        assistant_id = lines[0].strip()
        thread_id = lines[1].strip()
    print("Using existing Assistant and Thread IDs")
else:
    # Create an Assistant (Run Once)
    assistant = client.beta.assistants.create(
        name='Code Completion Assistant',
        instructions='As an expert code assistant, complete the provided code cell based on the instructions and previous code. Return only the code without any markdown, code fences, or explanations. The completed code should be enclosed in code fences (```python ... ```).',
        tools=[{"type": "code_interpreter"}],
        model=model,
    )
    assistant_id = assistant.id
    print("Assistant ID:", assistant_id)

    # Create a new Thread
    thread = client.beta.threads.create()
    thread_id = thread.id
    print("Thread ID:", thread_id)

    # Save assistant_id and thread_id to a file
    with open('assistant_thread_ids.txt', 'w') as f:
        f.write(f"{assistant_id}\n")
        f.write(f"{thread_id}\n")


Using existing Assistant and Thread IDs


In [4]:

print("Thread ID:", thread_id)       # Print the thread ID
print("Assistant ID:", assistant_id) # Print the assistant ID


# === Define the Thinking Function ===
def thinking(client, thread_id, run_id, sleep_interval=5):
    """
    Waits for a run to complete and prints the elapsed time.
    :param client: The OpenAI client object.
    :param thread_id: The ID of the thread.
    :param run_id: The ID of the run.
    :param sleep_interval: Time in seconds to wait between checks.
    """
    while True:
        try:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)  # Retrieve the current status of the run
            if run.completed_at:  # Check if the run has completed
                elapsed_time = run.completed_at - run.created_at  # Calculate elapsed time
                formatted_elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
                print(f"Run completed in {formatted_elapsed_time}")
                logging.info(f"Run completed in {formatted_elapsed_time}")  # Log the elapsed time

                # Get messages once Run is completed
                messages = client.beta.threads.messages.list(thread_id=thread_id)  # Retrieve all messages in the thread
                if messages.data:
                    last_message = messages.data[0]  # Assuming the latest message is first
                    # Depending on the Assistant API's response structure, adjust the access method
                    response = last_message.content.text.value if hasattr(last_message.content, 'text') else last_message.content
                    print(f"Assistant Response: {response}")
                else:
                    print("No messages found in the thread.")
                break
        except Exception as e:
            logging.error(f"An error occurred while retrieving the run: {e}")
            break
        logging.info("Waiting for run to complete...")
        time.sleep(sleep_interval)

# === Function to Read Jupyter Notebook ===
def read_notebook(file_path):
    """Read a Jupyter notebook file and parse it into a JSON object."""
    with open(file_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)
    return notebook


def extract_cells(notebook):
    """Extract markdown and code cells, removing large base64 image strings from markdown."""
    cells = notebook['cells']
    paired_cells = []  # List of tuples in the form (markdown_instructions, code_cell)

    current_markdown = []
    image_pattern = re.compile(r'!\[.*?\]\(data:image/(?:png|jpg|jpeg|gif);base64,[^\)]+\)')

    for cell in cells:
        if cell['cell_type'] == 'markdown':
            clean_source = re.sub(image_pattern, '', cell['source'])
            current_markdown.append(clean_source)
        elif cell['cell_type'] == 'code':
            paired_cells.append((current_markdown, cell))
            current_markdown = []  # Reset markdown for the next pair

    return paired_cells

# === Function to Filter Relevant Cells ===
def filter_relevant_cells(paired_cells):
    """
    Combine irrelevant code and markdown cells as instructions for the next relevant code cell 
    that includes any variation of the phrase 'start code here', case insensitive.
    """
    relevant_cells = []
    accumulated_instructions = []

    # Compile a regex pattern to match 'start code here' in a case-insensitive manner
    start_code_pattern = re.compile(r'start\s*code\s*here', re.IGNORECASE)

    for markdown_instructions, code_cell in paired_cells:
        combined_markdown = "\n".join(markdown_instructions)
        
        # Accumulate markdown instructions and code from irrelevant cells
        if not start_code_pattern.search(code_cell['source']):
            # Tag accumulated content to distinguish markdown and code
            accumulated_instructions.append(f"Markdown Instructions:\n{combined_markdown}")
            accumulated_instructions.append(f"Previous Code:\n{code_cell['source']}")
        else:
            # If the relevant marker is found, include accumulated instructions
            full_instructions = "\n\n".join(accumulated_instructions) + f"\n\nMarkdown Instructions:\n{combined_markdown}"
            relevant_cells.append(([full_instructions], code_cell))
            accumulated_instructions = []  # Reset the accumulated instructions

    return relevant_cells

# === Function to Generate Prompt ===
def generate_prompt(markdown_instructions, code):
    """
    Generate a prompt for the OpenAI API based on accumulated markdown instructions and 
    previous code cell content, clearly distinguishing their roles.
    """
    instructions_text = "\n".join(markdown_instructions)

    prompt = (
    f"Below are the instructions and previous code:\n\n"
    f"{instructions_text}\n\n"
    f"Current Code Cell:\n{code}\n\n"
    f"Complete the code in the 'Current Code Cell' based on the instructions. "
    f"Return only the code itself, without any markdown, explanations, or code fences."
    )

    return prompt

def complete_code(client, assistant_id, model, prompt, i):
    """
    Completes code based on the provided prompt using OpenAI's Assistant API.
    """

    try:
        # Create a new thread for this interaction
        thread = client.beta.threads.create()
        thread_id = thread.id
        logging.info(f"Created new thread with ID: {thread_id} for cell {i}")

        # Send the User Message to the Assistant
        client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=prompt
        )
        logging.info("Message sent to the assistant.")

        # Run the Assistant
        run = client.beta.threads.runs.create(
            thread_id=thread_id,
            model=model,
            assistant_id=assistant_id,
            # Updated instructions to request code within code fences
            instructions="As an expert code assistant, complete the provided code cell based on the instructions and previous code. Return the completed code enclosed within code fences (```python ... ```) to indicate the code."
        )
        logging.info(f"Assistant run initiated with Run ID: {run.id}")

        # Wait for the Assistant to Complete the Run
        while True:
            try:
                run_status = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
                if run_status.status == 'failed':
                    logging.error(f"Run failed with error: {run_status.error}")
                    return ""
                if run_status.completed_at:
                    elapsed_time = run_status.completed_at - run_status.created_at
                    formatted_elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
                    print(f"Run completed in {formatted_elapsed_time}")
                    logging.info(f"Run completed in {formatted_elapsed_time}")
                    break
            except Exception as e:
                logging.error(f"An error occurred while retrieving the run: {e}")
                return ""
            logging.info("Waiting for run to complete...")
            time.sleep(5)  # Adjust sleep interval as needed

        # Retrieve the Assistant's Response
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        if messages.data:
            # Find the last assistant message
            for message in reversed(messages.data):
                if message.role == 'assistant':
                    last_message = message
                    break
            else:
                logging.error("No assistant messages found in the thread after the run.")
                return ""

            # Log the entire content for debugging
            logging.info(f"Assistant Response Content: {last_message.content}")

            # Extract and return whatever content is provided as the code
            completed_code = ''
            content_blocks = last_message.content  # This is a list of content blocks
            for block in content_blocks:
                logging.info(f"Processing content block: {block}")
                if block.type == 'text':
                    # Directly use the text content as the completed code
                    completed_code += block.text.value + '\n'
                elif block.type == 'code':
                    # If the assistant returns a code block, handle it similarly
                    completed_code += block.code.code.value + '\n'

            completed_code = completed_code.strip()
            return completed_code
        else:
            logging.error("No messages found in the thread after the run.")
            return ""
    except Exception as e:
        logging.error(f"An error occurred in complete_code: {e}")
        return ""


def update_notebook(notebook, paired_cells):
    """Update the notebook with completed code in code cells."""
    i = 0
    for markdown_instructions, code_cell in paired_cells:
        current_code = code_cell['source']
        prompt = generate_prompt(markdown_instructions, current_code)
        # Pass the index `i` to `complete_code`
        completed_code = complete_code(client, assistant_id, model, prompt, i)
        if completed_code:
            completed_code = re.sub(r'```(?:python)?\n(.*?)```', r'\1', completed_code, flags=re.DOTALL).strip()
            print(f"Completed code for cell {i}")
            logging.info(f"Completed code for cell {i}")
            code_cell['source'] = completed_code
        else:
            logging.warning(f"No completion received for cell {i}")
        i += 1

        
# === Function to Save the Updated Notebook ===
def save_notebook(notebook, original_path):
    """Save the updated notebook with a new name to avoid overwriting the original."""
    directory, original_filename = os.path.split(original_path)
    new_filename = f"updated_{original_filename}"
    new_path = os.path.join(directory, new_filename)
    with open(new_path, 'w', encoding='utf-8') as f:
        nbformat.write(notebook, f)
    print(f"Notebook saved as {new_path}")
    logging.info(f"Notebook saved as {new_path}")

# === Main Processing Function ===
def process_notebook(file_path):
    """Complete the entire process: read, filter, update, and save the notebook."""
    notebook = read_notebook(file_path)
    input("read notebook finished, Press Enter to continue...")
    paired_cells = extract_cells(notebook)
    input("cells extracted finished, Press Enter to continue...")
    relevant_cells = filter_relevant_cells(paired_cells)  
    input("cells filtered for relevance. Press Enter to continue...")
    update_notebook(notebook, relevant_cells)
    input("notebook updated with completions. Press Enter to continue...")
    save_notebook(notebook, file_path)




Thread ID: thread_mM15QTqqjsYi9ROnyhPjusli
Assistant ID: asst_VipgeC5YkRAKLKHl4Jbg3w86


In [6]:
# Example usage:
file_path = "jupyter.ipynb"
process_notebook(file_path)

2024-10-16 14:33:28,938 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-10-16 14:33:28,940 - INFO - Created new thread with ID: thread_n7HWLILb8EbaKnKTnTqb9qBC for cell 0
2024-10-16 14:33:29,190 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_n7HWLILb8EbaKnKTnTqb9qBC/messages "HTTP/1.1 200 OK"
2024-10-16 14:33:29,191 - INFO - Message sent to the assistant.
2024-10-16 14:33:29,906 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_n7HWLILb8EbaKnKTnTqb9qBC/runs "HTTP/1.1 200 OK"
2024-10-16 14:33:29,907 - INFO - Assistant run initiated with Run ID: run_1vvGk3SlYp38LwE9ojKXOnlj
2024-10-16 14:33:30,128 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_n7HWLILb8EbaKnKTnTqb9qBC/runs/run_1vvGk3SlYp38LwE9ojKXOnlj "HTTP/1.1 200 OK"
2024-10-16 14:33:30,129 - INFO - Waiting for run to complete...
2024-10-16 14:33:35,355 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_n7HWLILb8EbaKnKTnT

Run completed in 00:00:05


2024-10-16 14:33:35,564 - INFO - Assistant Response Content: [TextContentBlock(text=Text(annotations=[], value='class InputEmbeddings(nn.Module):\n\n    def __init__(self, d_model: int, vocab_size: int) -> None:\n        super().__init__()\n        self.d_model = d_model\n        self.vocab_size = vocab_size\n        ### START CODE HERE ###\n        self.embedding = nn.Embedding(vocab_size, d_model)  # Use the nn.Embedding function here, use vocab_size and d_model as arguments\n        ### END CODE HERE ###\n\n    def forward(self, x):\n        # (batch, seq_len) --> (batch, seq_len, d_model)\n        # Multiply by sqrt(d_model) to scale the embeddings according to the paper\n\n        ### START CODE HERE ###\n        x = self.embedding(x)  # apply the self.embedding layer\n        x = x * math.sqrt(self.d_model)  # multiply output of embedding layer with square root of d_model\n        ### END CODE HERE ###\n\n        return x'), type='text')]
2024-10-16 14:33:35,565 - INFO - Processi

Completed code for cell 0


2024-10-16 14:33:35,780 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-10-16 14:33:35,781 - INFO - Created new thread with ID: thread_LYIBNeNdGqtZzIg31HDBY5DV for cell 1
2024-10-16 14:33:40,511 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_LYIBNeNdGqtZzIg31HDBY5DV/messages "HTTP/1.1 200 OK"
2024-10-16 14:33:40,513 - INFO - Message sent to the assistant.
2024-10-16 14:33:41,130 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_LYIBNeNdGqtZzIg31HDBY5DV/runs "HTTP/1.1 200 OK"
2024-10-16 14:33:41,131 - INFO - Assistant run initiated with Run ID: run_Knoxho55XG0yozJKg9nyVkwf
2024-10-16 14:33:41,405 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_LYIBNeNdGqtZzIg31HDBY5DV/runs/run_Knoxho55XG0yozJKg9nyVkwf "HTTP/1.1 200 OK"
2024-10-16 14:33:41,407 - INFO - Waiting for run to complete...
2024-10-16 14:33:46,764 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_LYIBNeNdGqtZzIg31H

...

Run completed in 00:00:16


2024-10-16 14:35:03,647 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_VTQ875jTkDAdqw3Cm9lgpFtI/messages "HTTP/1.1 200 OK"
2024-10-16 14:35:03,649 - INFO - Assistant Response Content: [TextContentBlock(text=Text(annotations=[], value='def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:\n\n    ### START CODE HERE ###\n    # Create the embedding layers\n    src_embed = EmbeddingLayer(d_model, src_vocab_size)  # Create an embedding layer for the source language.\n    tgt_embed = EmbeddingLayer(d_model, tgt_vocab_size)  # Create an embedding layer for the target language.\n    ### END CODE HERE ###\n\n    # Create the positional encoding layers\n    ### START CODE HERE ###\n    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)  # Positional encoding for source\n    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

Completed code for cell 7


2024-10-16 15:02:45,999 - INFO - Notebook saved as updated_jupyter.ipynb


Notebook saved as updated_jupyter.ipynb
